# 🧠 Step 1: Install Required Libraries

In [1]:
!pip install -q "pyarrow<20.0.0" "rich<14" "google-cloud-bigquery-storage>=2.30.0,<3.0.0" \
    transformers==4.44.0 datasets==3.0.0 accelerate==0.33.0 evaluate==0.4.2 scikit-learn==1.5.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 66.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 86.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 82.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━

In [2]:
import transformers, datasets, torch
print(transformers.__version__)
print("✅ Environment ready for Hugging Face on Kaggle!")

4.44.0
✅ Environment ready for Hugging Face on Kaggle!


# 🧾 Step 2: Import Dependencies

In [3]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate

# ✅ Confirm GPU availability in Kaggle
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🔥 Using device: {device}")

2025-11-07 20:38:57.488594: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762547937.748201      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762547937.814038      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🔥 Using device: cuda


# 📥 Step 3 — Load and Inspect Dataset

In [4]:
# Load the IMDB sentiment dataset (binary classification: positive/negative)
from datasets import load_dataset

dataset = load_dataset("imdb")

# Check basic structure
print(dataset)
print(dataset["train"][0])


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and

# 🔤 Step 4 — Tokenize the Text

## We’ll use a small, fast model to keep Kaggle GPU within limits.

In [5]:
from transformers import AutoTokenizer

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenisation function
def tokenize_function(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

# Apply tokenization to dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets.set_format("torch")

# For quick run, take subset (optional)
small_train = tokenized_datasets["train"].shuffle(seed=42).select(range(4000))
small_test = tokenized_datasets["test"].shuffle(seed=42).select(range(2000))

print(small_train)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 4000
})


# 🧠 Step 5 — Load Model aur Define Evaluation Metrics

## Run cell 👇

In [6]:
from transformers import AutoModelForSequenceClassification

# Pre-trained DistilBERT model for sentiment classification (2 classes)
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Metric calculation function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# ⚙️ Step 6 — Define Training Arguments (Kaggle-Optimized)

## GPU safe configuration 👇

In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/kaggle/working/sentiment_results",
    run_name="sentiment_run_v1",  # ✅ unique name to silence WandB warning
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="/kaggle/working/logs",
    logging_steps=100,
    report_to="none",  # ✅ fully disable wandb & tensorboard logs
    fp16=torch.cuda.is_available(),  # GPU mixed precision
)

# 🧩 Step 7 — Trainer Object

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train,
    eval_dataset=small_test,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


# 🏋️ Step 8 — Train the Model

## Run cell 👇

In [14]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.352000,0.277020,0.883000,0.902311,0.859000,0.880123
2,0.194400,0.283006,0.892000,0.885069,0.901000,0.892963


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=500, training_loss=0.30547201919555667, metrics={'train_runtime': 161.6648, 'train_samples_per_second': 49.485, 'train_steps_per_second': 3.093, 'total_flos': 529869594624000.0, 'train_loss': 0.30547201919555667, 'epoch': 2.0})

# 📊 Step 9 — Evaluate Trained Model

In [15]:
results = trainer.evaluate()
print("📈 Evaluation Results:")
print(results)

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


📈 Evaluation Results:
{'eval_loss': 0.28300580382347107, 'eval_accuracy': 0.892, 'eval_precision': 0.8850687622789783, 'eval_recall': 0.901, 'eval_f1': 0.8929633300297324, 'eval_runtime': 11.4049, 'eval_samples_per_second': 175.364, 'eval_steps_per_second': 10.96, 'epoch': 2.0}


# 💬 Step 10 — Custom Text Sentiment Prediction

## text model predict 👇

In [16]:
# Custom text examples
texts = [
    "I absolutely loved this movie, it was emotional and inspiring!",
    "The plot was weak and the acting was horrible.",
    "It was an average experience, not great but not bad either."
]

tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(model.device)
outputs = model(**tokens)
predictions = outputs.logits.argmax(dim=1)

for text, pred in zip(texts, predictions):
    sentiment = "Positive 😊" if pred == 1 else "Negative 😞"
    print(f"{text}\n→ {sentiment}\n")


I absolutely loved this movie, it was emotional and inspiring!
→ Positive 😊

The plot was weak and the acting was horrible.
→ Negative 😞

It was an average experience, not great but not bad either.
→ Positive 😊



# 💾 (Optional) Step 11 — Save Model for Kaggle Output Folder

In [17]:
model.save_pretrained("/kaggle/working/sentiment_model")
tokenizer.save_pretrained("/kaggle/working/sentiment_model")
print("✅ Model saved to /kaggle/working/sentiment_model")

✅ Model saved to /kaggle/working/sentiment_model
